In [ ]:
import pandas as pd

from summer2 import CompartmentalModel
from summer2.parameters import Parameter, DerivedOutput

In [ ]:
seir_data = pd.read_csv(
    "./csvs/model_4_3a.csv",
    index_col=0,
)

In [ ]:
config = {
    "tot_popn": 1e5,
    "infous_0": 1.,
    "end_time": 18250.,
    "t_step": 1.,
    "prop_immune_0": 0.3,
}
compartments = ("Susceptible", "Pre-infectious", "Infectious", "Immune")
model = CompartmentalModel(
    times=(0, config["end_time"]),
    compartments=compartments,
    infectious_compartments=["Infectious"],
    timestep=config["t_step"],
)
model.set_initial_population(
    distribution={
        "Susceptible": config["tot_popn"] - config["infous_0"],
        "Infectious": config["infous_0"],
    }
)
r0 = Parameter("r0")
ave_infous = Parameter("ave_infous")
model.add_infection_frequency_flow(
    name="infection", 
    contact_rate=r0 / ave_infous,
    source="Susceptible", 
    dest="Pre-infectious",
)
model.add_transition_flow(
    name="progression", 
    fractional_rate=1. / Parameter("ave_preinfous"),
    source="Pre-infectious", 
    dest="Infectious",
)
model.add_transition_flow(
    name="recovery", 
    fractional_rate=1. / ave_infous,
    source="Infectious", 
    dest="Immune",
)
model.add_universal_death_flows(
    "universal_death",
    death_rate=1. / Parameter("life_expectancy") / 365.,
)
model.add_replacement_birth_flow(
    "births",
    "Susceptible",
)
parameters = {
    "r0": 13.,
    "ave_preinfous": 8.,
    "ave_infous": 7.,
    "life_expectancy": 70.,
}
model.run(parameters=parameters, solver="euler")
seir_values = model.get_outputs_df()
assert abs(seir_data - seir_values).max().max() < 1e-5